In [1]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [2]:
train_folder='/content/drive/MyDrive/dataset/train'
test_folder='/content/drive/MyDrive/dataset/test'
val_folder='/content/drive/MyDrive/dataset/valid'

normal_folder='/content/drive/MyDrive/dataset/test/normal'
adenocarcinoma_folder='/content/drive/MyDrive/dataset/test/adenocarcinoma'
large_cell_carcinoma_folder='/content/drive/MyDrive/dataset/test/large.cell.carcinoma'
squamous_cell_carcinoma_folder='/content/drive/MyDrive/dataset/test/squamous.cell.carcinoma'

In [3]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder


import tensorflow as tf
import tensorflow.keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,SpatialDropout2D,Activation,Flatten
from tensorflow.keras.layers import Conv2D,MaxPooling2D,GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import utils

print("Libraries Imported ")

#setting the image Size

IMAGE_SIZE=(350,350)

print("Reading training images from:",train_folder)
print("Reading Validation images from:",val_folder)

#Rescaling the data

train_datagen=ImageDataGenerator(rescale=1./255,horizontal_flip=True)
test_datagen=ImageDataGenerator(rescale=1./255)

#batch size represents at at time how many images will process
batch_size=8

train_generator=train_datagen.flow_from_directory(
    train_folder,
    target_size=IMAGE_SIZE,
    batch_size=batch_size,
    color_mode='rgb',
    class_mode="categorical")

validation_generator=test_datagen.flow_from_directory(
    test_folder,
    target_size=IMAGE_SIZE,
    batch_size=batch_size,
    color_mode='rgb',
    class_mode="categorical")


Libraries Imported 
Reading training images from: /content/drive/MyDrive/dataset/train
Reading Validation images from: /content/drive/MyDrive/dataset/valid
Found 613 images belonging to 4 classes.
Found 315 images belonging to 4 classes.


In [6]:
from tensorflow.keras.callbacks import ReduceLROnPlateau,EarlyStopping,ModelCheckpoint
learning_rate_reduction=ReduceLROnPlateau(monitor='loss',patience=5,verbose=2,factor=0.5,min_lr=0.00001)
early_stops=EarlyStopping(monitor='loss',min_delta=0,patience=6,verbose=2,mode='auto')
checkpointer=ModelCheckpoint(filepath='best_model.weights.h5',verbose=2,save_best_only=True,save_weights_only=True)

In [7]:
import tensorflow as tf
from tensorflow.keras.layers import GlobalAveragePooling2D,Dense,Input
from tensorflow.keras.models import Model

IMAGE_SIZE=(224,224)
OUTPUT_SIZE=4

input_layer=Input(shape=(*IMAGE_SIZE,3))

pretrained_model=tf.keras.applications.Xception(
    weights='imagenet',
    include_top=False,
    input_tensor=input_layer
)
pretrained_model.trainable=False

x=GlobalAveragePooling2D()(pretrained_model.output)
output_layer=Dense(OUTPUT_SIZE,activation='softmax')(x)

model=Model(inputs=input_layer,outputs=output_layer)

print("Pretrained model Used:")
pretrained_model.summary()

print("Final model Created:")
model.summary()

model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

83683744/83683744 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step
Pretrained model Used:


Model: "xception"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 224, 224, 3)    │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1_conv1 (Conv2D)     │ (None, 111, 111, 32)   │            864 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1_conv1_bn           │ (None, 111, 111, 32)   │            128 │ block1_conv1[0][0]     │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1_conv1_act          │ (None, 111, 111, 32)   │              0 │ block1_conv1_bn[0][0]  │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1_conv2 (Conv2D)     │ (None, 109, 109, 64)   │         18,432 │ block1_conv1_act[0][0] │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1_conv2_bn           │ (None, 109, 109, 64)   │            256 │ block1_conv2[0][0]     │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1_conv2_act          │ (None, 109, 109, 64)   │              0 │ block1_conv2_bn[0][0]  │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block2_sepconv1           │ (None, 109, 109, 128)  │          8,768 │ block1_conv2_act[0][0] │
│ (SeparableConv2D)         │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block2_sepconv1_bn        │ (None, 109, 109, 128)  │            512 │ block2_sepconv1[0][0]  │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block2_sepconv2_act       │ (None, 109, 109, 128)  │              0 │ block2_sepconv1_bn[0]… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block2_sepconv2           │ (None, 109, 109, 128)  │         17,536 │ block2_sepconv2_act[0… │
│ (SeparableConv2D)         │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block2_sepconv2_bn        │ (None, 109, 109, 128)  │            512 │ block2_sepconv2[0][0]  │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d (Conv2D)           │ (None, 55, 55, 128)    │          8,192 │ block1_conv2_act[0][0] │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block2_pool               │ (None, 55, 55, 128)    │              0 │ block2_sepconv2_bn[0]… │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization  

 Total params: 20,861,480 (79.58 MB)

 Trainable params: 0 (0.00 B)

...

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 224, 224, 3)    │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1_conv1 (Conv2D)     │ (None, 111, 111, 32)   │            864 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1_conv1_bn           │ (None, 111, 111, 32)   │            128 │ block1_conv1[0][0]     │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1_conv1_act          │ (None, 111, 111, 32)   │              0 │ block1_conv1_bn[0][0]  │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1_conv2 (Conv2D)     │ (None, 109, 109, 64)   │         18,432 │ block1_conv1_act[0][0] │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1_conv2_bn           │ (None, 109, 109, 64)   │            256 │ block1_conv2[0][0]     │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1_conv2_act          │ (None, 109, 109, 64)   │              0 │ block1_conv2_bn[0][0]  │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block2_sepconv1           │ (None, 109, 109, 128)  │          8,768 │ block1_conv2_act[0][0] │
│ (SeparableConv2D)         │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block2_sepconv1_bn        │ (None, 109, 109, 128)  │            512 │ block2_sepconv1[0][0]  │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block2_sepconv2_act       │ (None, 109, 109, 128)  │              0 │ block2_sepconv1_bn[0]… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block2_sepconv2           │ (None, 109, 109, 128)  │         17,536 │ block2_sepconv2_act[0… │
│ (SeparableConv2D)         │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block2_sepconv2_bn        │ (None, 109, 109, 128)  │            512 │ block2_sepconv2[0][0]  │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d (Conv2D)           │ (None, 55, 55, 128)    │          8,192 │ block1_conv2_act[0][0] │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block2_pool               │ (None, 55, 55, 128)    │              0 │ block2_sepconv2_bn[0]… │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization  

 Total params: 20,869,676 (79.61 MB)

 Trainable params: 8,196 (32.02 KB)

 Non-trainable params: 20,861,480 (79.58 MB)

In [8]:
pip install --upgrade tensorflow

In [11]:
IMAGE_SIZE=(224,224)

train_datagen=ImageDataGenerator(rescale=1./255,horizontal_flip=True)
test_datagen=ImageDataGenerator(rescale=1./255)

batch_size=8

train_generator=train_datagen.flow_from_directory(
    train_folder,
    target_size=IMAGE_SIZE,
    batch_size=batch_size,
    color_mode='rgb',
    class_mode="categorical")

validation_generator=test_datagen.flow_from_directory(
    test_folder,
    target_size=IMAGE_SIZE,
    batch_size=batch_size,
    color_mode='rgb',
    class_mode="categorical")

try:
  history=model.fit(
      train_generator,
      steps_per_epoch=train_generator.samples//batch_size,
      epochs=5,
      callbacks=[learning_rate_reduction,early_stops,checkpointer],
      validation_data=validation_generator,
      validation_steps=validation_generator.samples//batch_size
  )

  print("Final training accuracy =",history.history['accuracy'][-1])
  print("Final testing accuracy =",history.history['val_accuracy'][-1])

except Exception as e:
  print(f'An error occurred: {e}')

Found 613 images belonging to 4 classes.
Found 315 images belonging to 4 classes.
Epoch 1/5
76/76 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.5077 - loss: 1.0997
Epoch 1: val_loss improved from inf to 0.90535, saving model to best_model.weights.h5
76/76 ━━━━━━━━━━━━━━━━━━━━ 371s 5s/step - accuracy: 0.5084 - loss: 1.0986 - val_accuracy: 0.5769 - val_loss: 0.9054 - learning_rate: 0.0010
Epoch 2/5
 1/76 ━━━━━━━━━━━━━━━━━━━━ 1:58 2s/step - accuracy: 0.6250 - loss: 0.6403
Epoch 2: val_loss improved from 0.90535 to 0.59854, saving model to best_model.weights.h5
76/76 ━━━━━━━━━━━━━━━━━━━━ 3s 16ms/step - accuracy: 0.6250 - loss: 0.6403 - val_accuracy: 0.6667 - val_loss: 0.5985 - learning_rate: 0.0010
Epoch 3/5
76/76 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.7132 - loss: 0.7452
Epoch 3: val_loss did not improve from 0.59854
76/76 ━━━━━━━━━━━━━━━━━━━━ 223s 3s/step - accuracy: 0.7129 - loss: 0.7454 - val_accuracy: 0.5929 - val_loss: 0.8678 - learning_rate: 0.0010
Epoch 4/5
 1/76 ━━━━━━━━━━━